In [2]:
import os
import yaml
from ultralytics import YOLO
import random
import os
import shutil

In [14]:


def split_images(image_directory, output_train, output_val, output_test):
    """
    Split images into training, validation, and testing sets.
    Args:
        image_directory: Directory containing the images.
        output_train: Directory to store the training set.
        output_val: Directory to store the validation set.
        output_test: Directory to store the testing set.
    """
    # Create output directories
    os.makedirs(output_train, exist_ok=True)
    os.makedirs(output_val, exist_ok=True)
    os.makedirs(output_test, exist_ok=True)

    # Get a list of image filenames
    image_filenames = [filename for filename in os.listdir(image_directory) if filename.endswith(('.jpg', '.jpeg', '.png'))]
    
    # Set the seed for reproducibility
    random.seed(42)

    # Shuffle the image filenames
    random.shuffle(image_filenames)

    # Calculate split indices
    total_images = len(image_filenames)
    train_split = int(0.6 * total_images)
    val_split = int(0.8 * total_images)

    # Assign images to splits
    train_images = image_filenames[:train_split]
    val_images = image_filenames[train_split:val_split]
    test_images = image_filenames[val_split:]

    # Copy images to the corresponding directories
    for filename in train_images:
        shutil.copy(os.path.join(image_directory, filename), os.path.join(output_train, filename))
    
    for filename in val_images:
        shutil.copy(os.path.join(image_directory, filename), os.path.join(output_val, filename))
    
    for filename in test_images:
        shutil.copy(os.path.join(image_directory, filename), os.path.join(output_test, filename))

    print("Splitting completed successfully.")

# Example usage:
image_directory_path = "./dataset/sim_training_data/sim_data_capture"
output_train_path = "./dataset/train/images"
output_val_path = "./dataset/val/images"
output_test_path = "./dataset/test/images"
split_images(image_directory_path, output_train_path, output_val_path, output_test_path)


Splitting completed successfully.


In [9]:
def normalize_bbox(x1, y1, x2, y2, image_width, image_height):
    """
    Normalize bounding box coordinates.
    Args:
        x1, y1, x2, y2: Bounding box coordinates.
        image_width, image_height: Dimensions of the image.

    Returns:
        Normalized bounding box coordinates (x_center_normalized, y_center_normalized, bbox_width_normalized, bbox_height_normalized).
    """
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    x_center = x1 + bbox_width / 2
    y_center = y1 + bbox_height / 2

    x_center_normalized = x_center / image_width
    y_center_normalized = y_center / image_height
    bbox_width_normalized = bbox_width / image_width
    bbox_height_normalized = bbox_height / image_height

    return x_center_normalized, y_center_normalized, bbox_width_normalized, bbox_height_normalized


def convert_annotations_to_txt(input_file, output_directory):
    class_index_mapping = {"Green": 0, "Yellow": 1, "Red": 2}
    
    with open(input_file, 'r') as f:
        annotations_list = yaml.safe_load(f)

    for i, image_annotation in enumerate(annotations_list):
        filename = os.path.join(output_directory, f"{image_annotation['filename'].split('/')[-1].split('.')[0]}.txt")

        with open(filename, "w") as file:
            
            for annotation in image_annotation['annotations']:
                class_index = class_index_mapping.get(annotation['class'], -1)
                
                x1 = annotation['xmin']
                y1 = annotation['ymin']
                x2 = x1 + annotation['x_width']
                y2 = y1 + annotation['y_height']

                # Normalize the values using the correct image dimensions (800x600)
                x_center_normalized, y_center_normalized, bbox_width_normalized, bbox_height_normalized = normalize_bbox(x1, y1, x2, y2, image_width=800, image_height=600)
                
                file.write(f"{class_index} {x_center_normalized} {y_center_normalized} {bbox_width_normalized} {bbox_height_normalized}\n")

                #file.write(f"{class_index}\t{annotation['x_width']}\t{annotation['xmin']}\t{annotation['y_height']}\t{annotation['ymin']}\n")

        print(f"File {i + 1}: {filename} created successfully.")

input_file_path = "./dataset/sim_data_annotations.yaml"
output_directory_path = "./dataset/sim_training_data/labels"
convert_annotations_to_txt(input_file_path, output_directory_path)

File 1: ./dataset/sim_training_data/labels/left0003.txt created successfully.
File 2: ./dataset/sim_training_data/labels/left0011.txt created successfully.
File 3: ./dataset/sim_training_data/labels/left0012.txt created successfully.
File 4: ./dataset/sim_training_data/labels/left0013.txt created successfully.
File 5: ./dataset/sim_training_data/labels/left0014.txt created successfully.
File 6: ./dataset/sim_training_data/labels/left0015.txt created successfully.
File 7: ./dataset/sim_training_data/labels/left0016.txt created successfully.
File 8: ./dataset/sim_training_data/labels/left0017.txt created successfully.
File 9: ./dataset/sim_training_data/labels/left0018.txt created successfully.
File 10: ./dataset/sim_training_data/labels/left0019.txt created successfully.
File 11: ./dataset/sim_training_data/labels/left0020.txt created successfully.
File 12: ./dataset/sim_training_data/labels/left0021.txt created successfully.
File 13: ./dataset/sim_training_data/labels/left0022.txt crea

In [10]:
def copy_labels(image_directory, label_directory, output_directory, label_extension='.txt'):
    """
    Copy label files to another directory using the image file names.
    Args:
        image_directory: Directory containing the images.
        label_directory: Directory containing the label files.
        output_directory: Directory to store the copied label files.
        label_extension: Extension of the label files (default is '.txt').
    """
    os.makedirs(output_directory, exist_ok=True)

    image_filenames = [filename for filename in os.listdir(image_directory) if filename.endswith(('.jpg', '.jpeg', '.png'))]

    for image_filename in image_filenames:
        label_filename = os.path.splitext(image_filename)[0] + label_extension
        source_label_path = os.path.join(label_directory, label_filename)
        destination_label_path = os.path.join(output_directory, label_filename)

        if os.path.exists(source_label_path):
            shutil.copy(source_label_path, destination_label_path)
            print(f"Label file for {image_filename} copied successfully.")

# Example usage:
train_image_directory_path = "./dataset/train/images"
val_image_directory_path = "./dataset/val/images"
test_image_directory_path = "./dataset/test/images"

label_directory_path = "./dataset/sim_training_data/labels"

output_train_path = "./dataset/train/labels"
output_val_path = "./dataset/val/labels"

copy_labels(train_image_directory_path, label_directory_path, output_train_path)
copy_labels(val_image_directory_path, label_directory_path, output_val_path)


Label file for left0435.jpg copied successfully.
Label file for left0529.jpg copied successfully.
Label file for left0623.jpg copied successfully.
Label file for left0014.jpg copied successfully.
Label file for left0575.jpg copied successfully.
Label file for left0413.jpg copied successfully.
Label file for left0550.jpg copied successfully.
Label file for left0609.jpg copied successfully.
Label file for left0625.jpg copied successfully.
Label file for left0617.jpg copied successfully.
Label file for left0469.jpg copied successfully.
Label file for left0507.jpg copied successfully.
Label file for left0015.jpg copied successfully.
Label file for left0584.jpg copied successfully.
Label file for left0602.jpg copied successfully.
Label file for left0509.jpg copied successfully.
Label file for left0481.jpg copied successfully.
Label file for left0503.jpg copied successfully.
Label file for left0044.jpg copied successfully.
Label file for left0024.jpg copied successfully.
Label file for left0

In [ ]:
#Train the model using YOLOv8
model = YOLO("yolov8s.pt")
model.train(data="./data.yaml", epochs=25, lr0= 0.005, iou=0.5, optimizer='AdamW')
metrics = model.val()

New https://pypi.org/project/ultralytics/8.1.6 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.230 🚀 Python-3.8.6 torch-2.1.2+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=./data.yaml, epochs=25, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

train: Scanning /mnt/irisgpfs/users/lweber/traffic_ligth_detector/dataset/train/labels... 165 images, 25 backgrounds, 0 corrupt: 100%|██████████| 168/168 [00:00<00:00, 645.54it/s]

train: New cache created: /mnt/irisgpfs/users/lweber/traffic_ligth_detector/dataset/train/labels.cache



/mnt/irisgpfs/users/lweber/jupyter_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 7, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /mnt/irisgpfs/users/lweber/traffic_ligth_detector/dataset/val/labels... 56 images, 7 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<00:00, 549.48it/s]

val: New cache created: /mnt/irisgpfs/users/lweber/traffic_ligth_detector/dataset/val/labels.cache


...


25 epochs completed in 0.047 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 22.5MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.0.230 🚀 Python-3.8.6 torch-2.1.2+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.02it/s]


                   all         56        141       0.99      0.993      0.992      0.777
                 Green         56         32      0.997          1      0.995      0.795
                Yellow         56         15      0.983          1      0.995      0.833
                   Red         56         94      0.989      0.979      0.986      0.704
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train7
Ultralytics YOLOv8.0.230 🚀 Python-3.8.6 torch-2.1.2+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /mnt/irisgpfs/users/lweber/traffic_ligth_detector/dataset/val/labels.cache... 56 images, 7 backgrounds, 0 corrupt: 100%|██████████| 56/56 [00:00<?, ?it/s]
/mnt/irisgpfs/users/lweber/jupyter_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 7, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.79it/s]


                   all         56        141       0.99      0.993      0.992       0.78
                 Green         56         32      0.997          1      0.995      0.798
                Yellow         56         15      0.983          1      0.995      0.833
                   Red         56         94      0.989      0.979      0.986      0.709
